In [2]:
from rdflib import Graph, Namespace, URIRef
from typing import Optional
from pydantic import BaseModel
from api.src.function.loc.getType import GetType
from pyfuseki import FusekiUpdate
import httpx

In [10]:
au_update = FusekiUpdate('http://localhost:3030', 'authorities')

r = au_update.run_sparql('DROP DEFAULT')
r.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [3]:
authority = 'http://id.loc.gov/authorities/names/n80002329'
url = f'http://localhost:8000/authorities/import/loc/agents?authority={authority}'
r = httpx.get(url)
r


<Response [200 OK]>

In [ ]:
r.json()

In [5]:
from api.src.schemas.authorities.authority import Variant, AdminMetadata, Element
class Uri(BaseModel):
    value: str
    label: str
    base: Optional[str]

class Affiliation(BaseModel):
    organization: Uri
    affiliationStart: str
    affiliationEnd: Optional[str]
    
class Agents(BaseModel):
    type: str 
    adminMetadata: AdminMetadata 
    elementList: list[Element]
    fullerName: Optional[Element]
    birthDate: Optional[str]
    birthPlace: Optional[str]
    deathDate: Optional[str]
    hasAffiliation: Optional[list[Affiliation]]
    occupation: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]
    subjectOf: Optional[list[Uri]]
    contributorOf: Optional[list[Uri]]
    isMemberOfMADSCollection: str
    

In [6]:
request = Agents(**r.json())
request

Agents(type='PersonalName', adminMetadata=AdminMetadata(assigner='http://id.loc.gov/vocabulary/organizations/dlc', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 5, 25), descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 5, 25, 9, 6, 30), identifiedBy=[IdentifiedBy(type='Lccn', assigner='http://id.loc.gov/vocabulary/organizations/dlc', value='n80002329')], status=Status(value='mstatus:new', label='novo')), elementList=[Element(type='FullNameElement', elementValue=Label(value='Machado de Assis,', lang=None)), Element(type='DateNameElement', elementValue=Label(value='1839-1908', lang=None))], fullerName=Element(type='PersonalName', elementValue=Label(value='Joaquim Maria Machado', lang=None)), birthDate='1839-06-21', birthPlace='Rio de Janeiro (Brazil)', deathDate='1908-09-29', hasAffiliation=[Affiliation(organization=Uri(value='http://id.

In [2]:
authority = 'http://id.loc.gov/authorities/names/n80002329'
graph = Graph()
graph.parse(f'{authority}.rdf')

<Graph identifier=N31c39298d6004c578ff4fa30d848ebce (<class 'rdflib.graph.Graph'>)>

In [180]:
def getElementList(graph, authority, obj):
    
  qElementList = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    SELECT ?type ?value 
    WHERE  {{
    <{authority}> madsrdf:elementList ?elementList .
    ?elementList rdf:rest* ?rest .
    ?rest rdf:first ?first .
    ?first rdf:type ?type  .
    ?first madsrdf:elementValue ?value
      }}"""
  r = graph.query(qElementList)
  bindings = r.bindings
  elementList = list()

  for i in bindings:
    element = {
            "type": i.get('type').split("#")[1],
          "elementValue": {
            "value":  i.get('value').toPython(),
            "lang": i.get('lang')
          }
        }
    elementList.append(element)
  obj['elementList'] = elementList

  return obj 

def getFullerName(graph, authority, obj):
    
  qFullerName = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    SELECT ?type ?value 
    WHERE  {{
    <{authority}> madsrdf:fullerName ?fullerName .
      ?fullerName rdf:type ?type .
      ?fullerName rdfs:label ?value
      }}"""
  r = graph.query(qFullerName)
  bindings = r.bindings
  if len(bindings) > 0:
    result = bindings[0]
    element = {
              "type": result.get('type').split("#")[1],
            "elementValue": {
              "value": result.get('value').toPython(),
              "lang": result.get('lang')
            }
          }
    obj['fullerName'] = element
    
  return obj

def GetUri(authority, graph, metadata, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:{metadata} ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
            url = i.get('value').toPython()
            base = url.split("//")[1].split("/")[0]
            uri = {
                   "value": url,
                   "base": base,
                   "label": i.get('label').value }
            external.append(uri)
        obj[metadata] = external
        
    return obj

def GetVariant(authority, graph, obj):
    
  q = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  SELECT ?typeVariant ?elementList ?elementValue ?typeElement 
  WHERE  {{      	
  <{authority}> madsrdf:hasVariant ?variant .
                  ?variant rdf:type ?typeVariant .
                  ?variant madsrdf:elementList ?elementList .
                  ?elementList rdf:rest* ?node .
                    ?node rdf:first ?e .
                    ?e madsrdf:elementValue ?elementValue .
                  ?e rdf:type ?typeElement .
                  FILTER ( ?typeVariant != madsrdf:Variant )
                  }}"""
  result = graph.query(q)
  
  if len(result.bindings) > 0:
    d = {}
    for i in result.bindings:
        bn = i.get('elementList').n3()
        typeVariant = i.get('typeVariant').split("#")[-1]
        elementValue = i.get('elementValue').toPython()
        tipo = i.get('typeElement').split("#")[-1]
        element = {'type': tipo, 'elementValue': {'value': elementValue}}
        
        if bn in d.keys():
            d[bn]['elementList'].append(element)
        else:
            d[bn] = {'type': typeVariant, 'elementList': [element]}
    variants = list(d.values())
    obj['hasVariant'] = variants
    return obj
  else:
     return obj

def GetDate(rwo, typeDate, graph, obj):

    q = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    SELECT ?date
    WHERE  {{      	
    <{rwo}> madsrdf:{typeDate} ?date . }}"""
    result = graph.query(q)

    if len(result.bindings) > 0:
        [birthDate] = result.bindings
        obj[typeDate] = birthDate.get('date').n3().split('"')[1]
    return obj

def GetBirthPlace(rwo, graph, obj):

    q = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE  {{      	
    <{rwo}> madsrdf:birthPlace ?birthPlace .
  ?birthPlace rdfs:label ?label }}"""
    result = graph.query(q)

    if len(result.bindings) > 0:
        [birthPlace] = result.bindings
        obj['birthPlace'] = birthPlace.get('label').toPython()

    return obj

def GetAffiliation(rwo, graph, obj):

    q = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    SELECT ?organization ?labelOrg ?start ?end
    WHERE  {{      	
    <{rwo}> madsrdf:hasAffiliation ?hasAffiliation .
  ?hasAffiliation madsrdf:organization ?organization .
  ?organization madsrdf:authoritativeLabel ?labelOrg .
  ?hasAffiliation madsrdf:affiliationStart ?start .
  OPTIONAL {{ ?hasAffiliation madsrdf:affiliationEnd ?end  }} }}"""
    result = graph.query(q)
    
    if len(result.bindings) > 0:
        affiliations = list()
        for i in result.bindings:
          # organization
          orgUri = i.get('organization').toPython()
          orgBase = orgUri.split('//')[1].split("/")[0]
          organization = {'value': orgUri, 'label': i.get('labelOrg').toPython(), 'base': orgBase}
          affiliation = {'organization': organization,
                         'affiliationStart': i.get('start').toPython(),
                         'affiliationEnd': i.get('end').toPython()}
          affiliations.append(affiliation)
        obj['hasAffiliation'] = affiliations

    return obj

def GetOccuption(rwo, graph, obj):

    q = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    SELECT ?occupation ?label
    WHERE  {{      	
    <{rwo}> madsrdf:occupation ?occupation .
   ?occupation madsrdf:authoritativeLabel ?label  }} """
    result = graph.query(q)
    if len(result.bindings) > 0:
       occupations = list()
       for i in result.bindings:
        value = i.get('occupation').toPython()
        base = value.split('//')[1].split("/")[0]
        occupation = {'value': value,
                     'label': i.get('label').value,
                     'base': base}
        occupations.append(occupation)
    obj['occupation'] = occupations
    
    return obj


token = authority.split("/")[-1]
rwo = f'http://id.loc.gov/rwo/agents/{token}'
obj = GetOccuption(rwo, graph, {})
obj

{'occupation': [{'value': 'http://id.loc.gov/authorities/subjects/sh85092863',
   'label': 'Novelists',
   'base': 'id.loc.gov'},
  {'value': 'http://id.loc.gov/authorities/subjects/sh85103733',
   'label': 'Poets',
   'base': 'id.loc.gov'},
  {'value': 'http://id.loc.gov/authorities/subjects/sh85039342',
   'label': 'Dramatists',
   'base': 'id.loc.gov'},
  {'value': 'http://id.loc.gov/authorities/subjects/sh85034154',
   'label': 'Critics',
   'base': 'id.loc.gov'},
  {'value': 'http://id.loc.gov/authorities/subjects/sh85108733',
   'label': 'Public officers',
   'base': 'id.loc.gov'}]}

In [181]:
def ParserAgents(graph, authority):
    # Type
    tipo = GetType(graph, authority)

    # adminMetadata
    adminMetadata = {
      "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
      "identifiedBy": [ {
         "type": "Lccn",
          "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
          "value": authority.split('/')[-1]        
      }]}
    
    obj = {
     "type": tipo,
      "adminMetadata": adminMetadata,
      "isMemberOfMADSCollection": f'http://bibliokeia.com/authorities/{tipo}/'}
    
    # ElementList
    obj = getElementList(graph, authority, obj)
    
    # fullerName
    obj = getFullerName(graph, authority, obj)

    # hasCloseExternalAuthority
    obj = GetUri(authority, graph, "hasCloseExternalAuthority", obj)

    # hasExactExternalAuthority
    obj = GetUri(authority, graph, "hasExactExternalAuthority", obj)

    # Variant
    obj = GetVariant(authority, graph, obj)

    # RWO
    token = authority.split("/")[-1]
    rwo = f'http://id.loc.gov/rwo/agents/{token}'
    # BirthDate
    obj = GetDate(rwo, 'birthDate', graph, obj)
    # deathDate
    obj = GetDate(rwo, 'deathDate', graph, obj)
    # BirthPlace
    obj = GetBirthPlace(rwo, graph, obj)
    # Affiliation
    obj = GetAffiliation(rwo, graph, obj)
    # Occuptions
    obj = GetOccuption(rwo, graph, obj)
    
    return obj



obj = ParserAgents(graph, authority)
request = Agents(**obj)
request.hasAffiliation

[Affiliation(organization=Uri(value='http://id.loc.gov/authorities/names/n82059239', label='Academia Brasileira de Letras', base='id.loc.gov'), affiliationStart='1897', affiliationEnd='1908')]

In [177]:
obj['occupation']

KeyError: 'occupation'